In [ ]:
#Отобрать аэропорты Австралии, из которых можно улететь за пределы Австралии
#Посчитать для каждого аэропорта сколько и для какого города я могу улететь
#Найти несовпадения табличек routes и airports
#Упорядочить страны по алфавиту
#Сгруппировать страны по континентам

In [ ]:
import numpy as np
import pandas as pd
from numpy import nan as NA

In [ ]:
#Подгружаем данные

airport = pd.read_csv('airports.csv', names = ['ID', 'Name', 'City', 'Country', 'IATA', 'ICAO', 'Latitude', 'Longitude', 'Altitude', 'Timezone', 'DST', 'Tz database time zone', 'Type', 'Source'])
routes = pd.read_csv('routes.csv', names = ['Airline', 'AirlineID', 'SourceAirport', 'Source Airport ID', 'DestinationAirport', 'Destination Airport ID', 'Codeshare', 'Stops', 'Equipment' ])

In [ ]:
#Отбираем из таблички airport аэропорт Австралии
#Важно! К Австралии относится не только сам остров Австралия, но и внешние территории, из них на 3 островах есть аэропорты
#В табличке airport в колонке country для них не Australia, а их названия: 
#Norfolk Island, Cocos (Keeling) Islands, Christmas Island
#Поэтому отбираем их тоже

australia = ['Australia', 'Norfolk Island', 'Cocos (Keeling) Islands', 'Christmas Island']
aus_air = airport[airport['Country'].isin(australia)]

In [ ]:
#Проверяем есть ли Na в IATA
icao_air = aus_air[aus_air['IATA']=='\\N']
icao_air

In [ ]:
#Есть ли IATA не длины 3
aus_air[aus_air['IATA'].str.len()!=3]

In [ ]:
#Na есть, но для них для всех есть ID и ICAO, проверим есть ли в routes по ним данные
#Проверить по ID
aus_id = np.array(icao_air['ID'].unique())
aus_id2 = np.array(['{:.0f}'.format(x) for x in aus_id])
routes[routes['Source Airport ID'].isin(aus_id2)]

#С такими ID нет

In [ ]:
#Проверка по ICAO
#В routes вообще нет  4 буквенных кодов
routes[routes['SourceAirport'].str.len()==4]

In [ ]:
routes[routes['DestinationAirport'].str.len()==4]

In [ ]:
#Поэтому можем заменить их на Nan и рабоать дальше без них
aus_air2 = aus_air.replace('\\N',NA)
aus_air2 = aus_air2[aus_air2['IATA'].str.len()==3]
aus_air2.count()

In [ ]:
#Отберем IATA для австралийских аэропортов, чтобы по ним обобрать маршруты
iata = np.array(aus_air2['IATA'].unique())
iata

In [ ]:
#Отбираем маршруты
aus_abroad = routes[routes['SourceAirport'].isin(iata) & -routes['DestinationAirport'].isin(iata)]

#Коды IATA австралийских аэропортов, из которых можно улететь в другие страны
aus_abroad['SourceAirport'].unique()

In [ ]:
#Направления куда летают
aus_abroad['DestinationAirport'].unique()

In [ ]:
#Количество
len(aus_abroad['DestinationAirport'].unique())

In [ ]:
air = pd.DataFrame(airport, columns=['IATA','Country'])
air = air.rename(columns = {'IATA': 'DestinationAirport'})
aus = aus_abroad.merge(air, on = 'DestinationAirport', how = 'left')

#Cлили таблички, проверим все ли страны подтянулись
aus['Country'].unique()


In [ ]:
aus.count()

In [ ]:
aus[aus['Country'].isna()]

In [ ]:
airport[airport['IATA']=='ZBL']
#В табличке airport нет этого аэропорта
#ZBL это австралийский аэропорт, поэтому проверили летают ли из него в другие страны

In [ ]:
routes[(routes['SourceAirport']=='ZBL') & -routes['DestinationAirport'].isin(iata)]
#Из него в другие страны не летают, но выкинем его из таблички aus, т.к. в Destination не должны быть австралийские аэропорты

In [ ]:
airport[airport['IATA']=='DOH']
#Это аэропорт Дохи(Катар) и его тоже не оказалось в табличке airport

In [ ]:
aus.loc[aus['DestinationAirport'] == 'DOH', 'Country'] = 'Qatar'
aus

In [ ]:
aus_new = aus[aus['DestinationAirport'] != 'ZBL']
aus_new.count()

In [ ]:
final = aus_new.pivot_table(values = 'Airline', index=['Country','DestinationAirport'],  columns='SourceAirport', aggfunc='count')
final.sort_index()

In [ ]:
#Объеденим по континентам, сначала посмотрим какой список стран
aus_new['Country'].unique()

In [ ]:
#Соотнесем с континентами
continent = {'Singapore':'Asia', 'China':'Asia', 'New Zealand':'Oceania', 'United States':'North America', 'Canada':'North America',
       'India':'Asia', 'Hong Kong':'Asia', 'United Arab Emirates':'Asia', 'Brunei':'Asia', 'Taiwan':'Asia',
       'Papua New Guinea':'Oceania', 'Malaysia':'Asia', 'Thailand':'Asia', 'Indonesia':'Asia', 'Fiji':'Oceania',
       'Solomon Islands':'Oceania', 'Japan':'Asia', 'South Korea':'Asia', 'Chile':'South America', 'Mauritius':'Africa',
       'Vanuatu':'Oceania', 'Cook Islands':'Oceania', 'Nauru':'Oceania', 'Philippines':'Asia', 'New Caledonia':'Oceania',
       'Qatar':'Asia', 'East Timor':'Asia', 'Vietnam':'Asia', 'South Africa':'Africa', 'Guam':'Oceania', 'Samoa':'Oceania',
       'Tonga':'Oceania'}

In [ ]:
aus_new['Continent'] = aus_new['Country'].map(continent)

In [ ]:
aus_new

In [ ]:
final2 = aus_new.pivot_table(values = 'Airline', index=['Continent', 'Country','DestinationAirport'],  columns='SourceAirport', aggfunc='count')
final2.sort_index()